In [ ]:
#Python standard library imports
import datetime as dt
import pathlib
import re
#Scientific Python ecosystem imports
import pandas as pd
#Text mining packages
import nltk as nltk
from whoosh.lang.porter import stem

#The local library where we will move the functions
#we design and condense from the original keyword_count.py
from polmap.polmap import prepare_keywords

In [ ]:
#Set Options for pandas visualization
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_seq_item', None)
pd.set_option('precision', 3)

In [ ]:
########### 2) read in all keywords #MM this could be moved below after the pdf conversion and before the counting, to have a little bit more of a flow:
#e.g. 1) create global variable, 2) convert doc to text 3) load keywords, lemmatize keywords and text, 4) count keys in text 5) save output

#MM_ set global viariables for text mining.

##process words - remove stopwords, stemming, lemmatizing

#set stop words
#stop_words = set(nltk.corpus.stopwords.words("english"))
#remove all from stop_words to keep in keywords
#stop_words.remove("all")

#MM_process and edit keys

keys = pd.read_excel('keys_from_RAKE-GBV_DB_SB_v3.xlsx', sheet_name= 'Sheet1' )
keys2 = pd.read_excel('keys_from_RAKE-GBV_DB_SB_v3.xlsx', sheet_name= 'Sheet1' )
#read in 1 row with keys per target
# split row by ; - series of list of strings

keys['Keys'] = keys['Keys'].str.split(pat = ";")

#lemmatizer = WordNetLemmatizer()
for i in range(0, len(keys['Keys'])): #applymap instead of 3 nested for loops?
    #print(keys['Keys'][i])
    for j in range(0,len(keys['Keys'][i])):
                # keys['Keys'][i][j] = word_tokenize(keys['Keys'][i][j])
        keys['Keys'][i][j] = [re.sub(r"[^a-zA-Z-]+", '', t.lower().strip()) for t in keys['Keys'][i][j].split()]
        #add whitespaces to words
        keys['Keys'][i][j] = [word.center(len(word)+2) for word in keys['Keys'][i][j]]
        #transform rd back to R&D for later detection
        keys['Keys'][i][j] = [w.replace(" rd ", "R&D") for w in keys['Keys'][i][j]]
        #remove words > 2
        keys['Keys'][i][j] = [word for word in keys['Keys'][i][j] if len(word) > 2 or word == "ph"]
        # remove '
        # keys['Keys'][i][j] = [s.replace('\'', '') for s in keys['Keys'][i][j]]
        # remove whitespaces
        keys['Keys'][i][j] = [x.strip(' ') for x in keys['Keys'][i][j]]
        # add special char to prevent aids from being stemmed to aid
        keys['Keys'][i][j] = [w.replace("aids", "ai&ds&") for w in keys['Keys'][i][j]]
        keys['Keys'][i][j] = [w.replace("productivity", "pro&ductivity&") for w in keys['Keys'][i][j]]
        keys['Keys'][i][j] = [w.replace("remittances", "remit&tance&") for w in keys['Keys'][i][j]]
        #stem words
        keys['Keys'][i][j] = [stem(word) for word in keys['Keys'][i][j] if not word in stop_words if word != "aids"]
        # remove special char for detection in text
        keys['Keys'][i][j] = [w.replace("ai&ds&", "aids") for w in keys['Keys'][i][j]]
        keys['Keys'][i][j] = [w.replace("pro&ductivity&", "productivity") for w in keys['Keys'][i][j]]
        keys['Keys'][i][j] = [w.replace("remit&tance&", "remittance") for w in keys['Keys'][i][j]]
        # lemmatizing words
        # keys['Keys'][i][j] = [lemmatizer.lemmatize(word) for word in keys['Keys'][i][j] if not word in stop_words]
        #merge back together to 1 string
        keys['Keys'][i][j] = ' '.join(keys['Keys'][i][j])
        #add leading and trailing whitespace
        keys['Keys'][i][j] = " " + keys['Keys'][i][j] + " "
        print(keys['Keys'][i][j])


#keys.head(170)
######################################

In [ ]:
keys.head()

In [ ]:
stop_words = set(nltk.corpus.stopwords.words("english"))
#remove all from stop_words to keep in keywords
stop_words.remove("all")
keys2['Keys']=keys2['Keys'].apply(lambda x: prepare_keywords(x, stop_words))
keys2.head()
